In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/home/aliche/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [3]:
%matplotlib
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import fnmatch,os
import itertools
import statsmodels.formula.api as sm
from scipy.stats import pearsonr,ttest_ind,ranksums,linregress,ks_2samp,entropy,mannwhitneyu
import pandas as pd
import seaborn as sns
from signale.tools import locate
import statsmodels.api as sm
from matplotlib.gridspec import GridSpec
from scipy.cluster.hierarchy import fcluster,linkage,dendrogram
from scipy.io import savemat,loadmat
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
###100ms
les100rn = sorted([os.path.join(item[0],item[1])for item in locate('*100ms-vis.lsn','../PCA_MEC/gammas/')])
cnt100rn = sorted([os.path.join(item[0],item[1])for item in locate('*100ms-vis.cnt','../PCA_MEC/gammas/')])
###Theta
lesThrn = sorted([os.path.join(item[0],item[1])for item in locate('*-theta-vis.lsn','../PCA_MEC/gammas/')])
cntThrn = sorted([os.path.join(item[0],item[1])for item in locate('*-theta-vis.cnt','../PCA_MEC/gammas/')])
spikeDATApath = '/home/aliche/dataWork/ali/MECLesion_SleepData/'
dataFolder    = '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/' 
spkFiles = sorted([os.path.join(item[0],item[1])for item in locate('FiringUnits.spk',spikeDATApath)])
trajDBs = sorted([os.path.join(item[0],item[1])for item in locate('linearTrack*.db',dataFolder) if item[0].find('trajectory')>0])
trajFLs = sorted([os.path.join(item[0],item[1])for item in locate('*.traj',spikeDATApath)])

In [92]:
for pth in les100rn:
    Thpth = pth.replace('100ms','theta')
    info = pth.split('/')[-1].split('-')
    anID = info[0]
    Dday   = info[1]
    DD = Dday[0]+Dday[-1]+'.db'
    spkPth = [spkpth  for spkpth in spkFiles if (spkpth.find(Dday)>0 and spkpth.find(anID)>0) ][0]
    trjPth = [trj  for trj in trajFLs if (trj.find(Dday)>0 and trj.find(anID)>0) ][0]
    dbPth = [db  for db in trajDBs if (db.find(DD)>0 and db.find(anID)>0) ][0]
    Nslices = 1 #number of slices that a theta cycle should be cut
    parrentFolder = spkPth[:-15]
    stList = pd.read_pickle(spkPth)
    stList.changeTimeUnit('ms')
    traj   = pd.read_pickle(trjPth)
    #Interpolating trajectory
    t = traj.times
    It = np.arange(0,traj.times.size)
    Itp = np.arange(0,traj.times.size,0.15)
    tInterp = np.interp(Itp,It,t) #interpolated traj times
    
    sig100 = stList.id_list.size - pd.read_pickle(pth).shape[1]
    sigtet = stList.id_list.size - pd.read_pickle(pth.replace('100ms','theta')).shape[1]
    if Nslices==1:
        tt = pkl.load(open(parrentFolder+'thBins.npArr','rb'))
        bb = 100
    else:
        tt = pkl.load(open(parrentFolder+'4thBins.npArr','rb'))
        bb = 25
    pre,track,post  = [stList.time_slice(item[0],item[-1]) for item in tt[:3]]
    immo = stList.time_slice(tt[1][0],tt[1][-1])
    ExperimentSessions = [pre,track,immo,post]
    for sessID,sess in enumerate(ExperimentSessions):
        for _id,cell in enumerate(sess):
            if cell.spike_times.size ==0:
                print sessID,_id
                cell.spike_times = np.append(cell.spike_times,sess.first_spike_time())
    
    #track.changeTimeUnit('s')
    #immo.changeTimeUnit('s')
    tdb = pd.read_pickle(dbPth)
    ####################################################################################
    for _idx_,cell in enumerate(track):
        #print '_________%s_________' %_idx_
        immSPKids = np.zeros_like(cell.spike_times)
        for _ii_ in range(tdb.shape[0]):
            immSPKids+= np.logical_and(cell.spike_times<tInterp[tdb[_ii_,0]]*1e3,cell.spike_times>tInterp[tdb[_ii_-1,1]]*1e3)
        immSPKids = immSPKids.astype(bool)
        immo[_idx_].spike_times = cell.spike_times[immSPKids]
        runSPKids = np.logical_not(immSPKids)
        track[_idx_].spike_times = cell.spike_times[runSPKids]
    immoTT = np.zeros_like(tt[1])
    for _ii_ in range(tdb.shape[0]):
        immoTT+= np.logical_and(tt[1] < tInterp[tdb[_ii_,0]]*1e3,tt[1] > tInterp[tdb[_ii_-1,1]]*1e3)
    immoTT = immoTT.astype(bool)
    #TT = [tt[0],tt[1][np.logical_not(immoTT)],tt[1][immoTT],tt[2]]
    TT = [tt[0],tt[1],tt[1],tt[2]]
    ###################################################################################
    print spkPth
    corrMatrices = []
    Qs = []
    eigVals = []
    eigVecs = []
    Ps = []
    #track.changeTimeUnit('ms')
    #immo.changeTimeUnit('ms')
    for sessID,sess in enumerate(ExperimentSessions):
        for _id,cell in enumerate(sess):
            if cell.spike_times.size ==0:
                print sessID,_id
                cell.spike_times = np.append(cell.spike_times,sess.t_start+np.random.randint(10,1000))
    for ii,item in enumerate(ExperimentSessions):
        print ii
        Q,C,l,V,P = item.PCA(bb)
        corrMatrices.append(C)
        Qs.append(Q)
        eigVals.append(l)
        eigVecs.append(V)
        Ps.append(P)
    #####PCA with time bins defined by theta periods
    ThetaCorrMatrices = []
    ThetaQs = []
    ThetaEigVals = []
    ThetaEigVecs = []
    ThetaPs = []
    for ii,item in enumerate(ExperimentSessions):
        print ii
        Q,C,l,V,P = item.PCA(bins=TT[ii])
        ThetaCorrMatrices.append(C)
        ThetaQs.append(Q)
        ThetaEigVals.append(l)
        ThetaEigVecs.append(V)
        ThetaPs.append(P)
    Gamma = []
    TGamma = []
    N = int(stList.id_list.size)
    for ii in range(4):
        G = []
        TG = []
        for jj in range(int(N)):
            G.append(eigVecs[1][jj].T.dot(corrMatrices[ii]).dot(eigVecs[1][jj]))
            TG.append(ThetaEigVecs[1][jj].T.dot(ThetaCorrMatrices[ii]).dot(ThetaEigVecs[1][jj]))
        Gamma.append(np.array(G))
        TGamma.append(np.array(TG))
        
    reactivationStrengths  = np.array([Gamma[0][sig100:],Gamma[2][sig100:],Gamma[3][sig100:]])
    TreactivationStrengths = np.array([TGamma[0][sigtet:],TGamma[2][sigtet:],TGamma[3][sigtet:]])
    fixedBinsName = str(100/Nslices)+'ms'
    if Nslices==4:
        thBinsName = '4theta'
    elif Nslices==1:
        thBinsName = 'theta'
    pkl.dump(reactivationStrengths, open('../PCA_MEC/DATA_REV/'+anID+'-'+DD+'-'+fixedBinsName+'-vis.lsn','wb'),pkl.HIGHEST_PROTOCOL)
    pkl.dump(TreactivationStrengths,open('../PCA_MEC/DATA_REV/'+anID+'-'+DD+'-'+thBinsName+'-vis.lsn','wb'),pkl.HIGHEST_PROTOCOL)

1 14
1 22
2 14
2 22
3 22
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-03-2015_Day1/parrentData/FiringUnits.spk
1 7
1 29
2 14
2 22
0
bin edges are caclulated with width 100 ms
1
bin edges are caclulated with width 100 ms
2
bin edges are caclulated with width 100 ms
3
bin edges are caclulated with width 100 ms
0
time bin edges are proovided by user.
1
time bin edges are proovided by user.
2
time bin edges are proovided by user.
3
time bin edges are proovided by user.
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-04-2015_Day2/parrentData/FiringUnits.spk
0
bin edges are caclulated with width 100 ms
1
bin edges are caclulated with width 100 ms
2
bin edges are caclulated with width 100 ms
3
bin edges are caclulated with width 100 ms
0
time bin edges are proovided by user.
1
time bin edges are proovided by user.
2
time bin edges are proovided by user.
3
time bin edges are proovided by user.
/home/aliche/dataWork/ali/MECLesion_SleepDat

/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/parrentData/FiringUnits.spk
2 3
0
bin edges are caclulated with width 100 ms
1
bin edges are caclulated with width 100 ms
2
bin edges are caclulated with width 100 ms
3
bin edges are caclulated with width 100 ms
0
time bin edges are proovided by user.
1
time bin edges are proovided by user.
2
time bin edges are proovided by user.
3
time bin edges are proovided by user.
0 10
1 10
2 10
3 10
/home/aliche/dataWork/ali/MECLesion_SleepData/Magda/Rat405/LinearTrack_0-0-0_Day1/parrentData/FiringUnits.spk
1 27
2 10
0
bin edges are caclulated with width 100 ms
1
bin edges are caclulated with width 100 ms
2
bin edges are caclulated with width 100 ms
3
bin edges are caclulated with width 100 ms
0
time bin edges are proovided by user.
1
time bin edges are proovided by user.
2
time bin edges are proovided by user.
3
time bin edges are proovided by user.


In [22]:
trajFLs = sorted([os.path.join(item[0],item[1])for item in locate('*.traj',spikeDATApath)])

In [46]:
#[trj  for trj in trajFLs if (trj.find(Dday)>0 and trj.find(anID)>0) ][0]
[spkpth  for spkpth in spkFiles if (spkpth.find(Dday)>0 and spkpth.find(anID)>0) ]

[]

In [64]:
info,Dday,anID

(['rat3661D1', '100ms', 'vis.cnt'], '100ms', 'rat3661D1')

In [227]:
pl.plot(TT[1],np.ones_like(TT[1]),'.')
pl.plot(TT[2],-1*np.ones_like(TT[2]),'.')

In [91]:
pl.imshow(C)

In [89]:
pl.scatter(Gamma[0],Gamma[1])
pl.scatter(Gamma[0],Gamma[2])
pl.scatter(Gamma[0],Gamma[3])

In [74]:
item.raster_plot()

In [72]:
pl.plot(TT[1],np.ones_like(TT[1]),'.')
pl.plot(TT[2],0.9*np.ones_like(TT[2]),'.')

In [76]:
cntThrn

['/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3661-Day3-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3839-Day1-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3839-Day2-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3839-Day4-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3839-Day5-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3906-Day1-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3906-Day2-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3906-Day3-theta-vis.cnt',
 '/home/aliche/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/PCA_MEC/DATA/Rat3906-Day4-theta-vis.cnt',
 '/home/aliche/ownCloud/Ipyt

In [36]:
spkPth

'/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3903/LinearTrack_05-12-16_Day4/parrentData/FiringUnits.spk'